# Shakeup scatterplots: Boxes, strings and things...
Almost all data tells a story, and competition shakeups are no different. Whilst looking through a collection of shakeup plots in the excellent notebook ["Meta Kaggle: Scatter Plot Competition Shake-up"](https://www.kaggle.com/jtrotman/meta-kaggle-scatter-plot-competition-shake-up) by [jtrotman](https://www.kaggle.com/jtrotman) it is hard not to notice various 'features' that appear again and again.
This is a small selection of examples, and it would be nice to explain what is going on in each case. This notebook is a "work in progress" and suggestions and insights are **more than welcome**, especially from people who participated in these competitions. Although the plots are interactive and each point corresponds to a specific kaggle user, the vast majority of the corresponding notebooks behind each of these points are private, or a `submission.csv` was uploaded, so it will require some detective work to understand the complete story behind each of the features. The three main features are **isoscore strings**, **blending boxes**, and the occasional **discontinuity**.


# General observations
| Region | Explanation |
| :--- | :--- |
| Top RHS | This region may correspond to low-scoring initial baselines |
| Bottom LHS | These are winners who made a fantastic submission |
| Top LHS | These people had a very bad shakeup|
| Bottom RHS | These people had a very good shakeup |

We shall see that the top RHS quadrant is often 'well behaved', especially in comparison to the bottom LHS quadrant. This region may be populated by baselines or fairly simple initial models. A suggestion as to what is going on in this tail end of the leaderboard has been made by [jtrotman](https://www.kaggle.com/jtrotman) in the comments section below, which I reproduce here:

> "...*the long tail with minimal shake. Competitions used to have a rule that you had to submit before the final week (and always at least one forum post by someone who didn't know that, but had put a lot of work in and was frustrated at being locked out!) With old competitions in particular there are many who submitted only the sample submission, just in case they wanted to join later. I guess it still happens to some extent...*"

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')
comps = pd.read_csv('../input/meta-kaggle/Competitions.csv')
COLOR_DICT = {0: 'deepskyblue', 1: 'gold', 2: 'silver', 3: 'chocolate'}
MEDAL_NAMES = np.asarray(["None", "Gold", "Silver", "Bronze"])
MEDAL_COLORS = dict(zip(MEDAL_NAMES, COLOR_DICT.values()))
def shakeupScatterplot(search_string):
    our_competition  = comps[comps['Title'].str.contains(search_string,na=False)]
    pd.set_option('display.max_columns', None)
    CompetitionId = our_competition["Id"].squeeze()
    CompetitionIndex = our_competition.index.values.astype(int)[0]
    all_teams = pd.read_csv('../input/meta-kaggle/Teams.csv')
    teams = all_teams[all_teams['CompetitionId']==CompetitionId]
    teams = teams.assign(Medal=teams.Medal.fillna(0).astype(int))
    row = comps.loc[CompetitionIndex]
    teams = teams.assign(Medal=MEDAL_NAMES[teams.Medal])
    fig = px.scatter(teams,
                 title='Shakeup plot for: ' + row.Title,
                 x='PublicLeaderboardRank',
                 y='PrivateLeaderboardRank',
                 hover_name='TeamName',
                 hover_data=[
                     'ScoreFirstSubmittedDate',
                     'LastSubmissionDate',
                     'PublicLeaderboardSubmissionId',
                     'PrivateLeaderboardSubmissionId',
                     'Medal',
                     'MedalAwardDate',
                 ],
                 color='Medal',
                 color_discrete_map=MEDAL_COLORS)
    fig.update_traces(marker=dict(size=4))
    #fig.update_layout(height=750, showlegend=False)
    fig.update_layout(showlegend=False)
    fig.show()
    return

# No shakeup and no surprises: WYSIWYG
This is the shakeup plot for the [Halite by Two Sigma](https://www.kaggle.com/c/halite) competition. Given the unusual format of this competition there is no traditional shakeup as such. However it does make for a nice *ideal case* plot. The `PublicLeaderboardRank` is equal to the `PrivateLeaderboardRank` and all points fall on the main diagonal:

In [ ]:
shakeupScatterplot("Halite")

# Isoscore 'strings'
Here we can see a large diffuse *blending box* (see the meaning of this below). Within the box we can see several strings whose trajectory parallels the main diagonal. We can also see in this particular competition that the strings are more or less piecewise continuous, but that is not generally the case.

In [ ]:
shakeupScatterplot("Bengali.AI Handwritten Grapheme Classification")

Strings are constituted from submissions that all have *exactly* the same score, forming contiguous leaderboard ranks, and quite probably originated from people who all submitted the same forked notebook output file. In other words, strings represent an *isoscore*. The first person to submit the notebook creates the head of the string situated to the bottom left, and obtains both the best Public and Private leaderboard rank, and the last person to submit the notebook is added to the end of the string situated at the top right. They were all subjected to the same shakeup so are all shifted either above or below the main diagonal, depending on how good the original *seed* notebook performed. We can clearly see at least five strings, so there were at least five notebooks that had a large influence on this competition.

# 'Blending' boxes
Here we can see an example of a '*blending box*' in the [Santander Customer Transaction Prediction](https://www.kaggle.com/c/santander-customer-transaction-prediction) competition:

In [ ]:
shakeupScatterplot("Santander Customer Transaction Prediction")

One can see that there were approximately 9000 participants in this competition and things seem 'normal' (leading diagonal with slight upward and downward shakeup) until one reaches the top $\approx$ 3000 participants in the public leaderboard. It is at this point that a box forms and all of the bronze medals and a good many of the silver medals spring from this region. 

Boxes generally tend to appear in the lower left quadrant (although in the OpenVaccine: COVID-19 competition [one appeared in the middle](https://www.kaggle.com/c/stanford-covid-vaccine/discussion/190101)), which is the area populated by submissions mostly made towards the end of the competition. Both the top and the bottom edges of the box are *diffuse quasi-isoscores*, and for that reason they are effectively almost horizontal rather than having the diagonal inclination of a truly contiguous isoscore string.
We can map out the scores at the corners of this particular box (parallelogram), which are roughly:

| Corner | Public score| Private score | Corner | Public score | Private score |
| :--- | :--- | :--- | :--- | :--- | :--- |
| Top LHS    | 0.9012 | 0.8997 | Top RHS    | 0.9008 | 0.8996 |
| Bottom LHS | 0.9013 | 0.9001 | Bottom RHS | 0.9009 | 0.8999 |


It can be seen that, despite the dramatic looking shakeup, this is *almost* an isoscore region, whose submissions were quite possibly based on notebooks such as ["Modified Naive Bayes - Santander - [0.899]"](https://www.kaggle.com/cdeotte/modified-naive-bayes-santander-0-899) (which was forked well over 300 times).
There is an element of symmetry here because if somebody shakes-*up*, then somebody else must shake-*down*.


I general I propose that the bottom edge is composed of similar scoring submissions based on a very good idea/notebook that was suggested towards the end of the competition, and many people simultaneously decided to go with that idea as their final submission.
I suspect that boxes are then filled in by people *blending* various different submission files together (or the same set of submission files but with different weights) resulting in the variability seen in the final Private score.
Unfortunately I suspect the top edge of boxes are composed of people who submitted a particularly dodgy blended notebook that was published late in the game and which looked good on paper but in reality was overfitting the Pulblic test data ...*Be Warned!*
# A discontinuity
Here is a clear example of a discontinuity:

In [ ]:
shakeupScatterplot("Google Landmark Retrieval 2020")

This is a section of the main diagonal that has blown apart, froming an *'isoscore region'*. In this example what we are seeing here in the range between the ranks of 254 (the seed notebook being the [organizers baseline model](https://www.kaggle.com/camaskew/baseline-submission)) and 474 is that everyone had the very same Public score of `0.27154`, and everyone apparently having the very same Private leaderboard score of `0.24382`. This should really be an isoscore string, however, seemingly randomly some people went up the Private leaderboard and others went down. This could have been decided by something even as small as a [round-off error](https://en.wikipedia.org/wiki/Round-off_error) in the submission score calculation. Indeed, a close inspection of the meta kaggle `Submissions.csv` file shows that the `PublicScoreFullPrecision` of `0.271545` resolves into a `PrivateScoreFullPrecision` of either `0.243829` or `0.243832` with no obviously discernible pattern. A similar discontinuity was also present in the [RSNA STR Pulmonary Embolism Detection
](https://www.kaggle.com/c/rsna-str-pulmonary-embolism-detection/discussion/194012) competition.
# And finally: Just plain crazy...
...the [M5 Forecasting - Accuracy](https://www.kaggle.com/c/m5-forecasting-accuracy) competition:

In [ ]:
shakeupScatterplot("M5 Forecasting - Accuracy")

### Any comments, criticisms, insights, and suggestions are more than welcome!